In [202]:
# Imports

import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
os.environ['QT_QPA_PLATFORM'] = 'xcb'
from scipy.ndimage import median_filter
from PIL import Image
from sklearn.cluster import DBSCAN
from skimage.morphology import erosion, square

In [ ]:
# Use bag file after converting it to csv using rosbag
# Replace 233 with the number of rows in the csv file

df = pd.read_csv('dvs-events.csv')

for i in range(0, 233):
    x = df.iloc[i,7]
    df_1 = pd.DataFrame([x], columns=['events'])
    df_1.to_csv(f'events/df_{i}.csv', index=False)

In [178]:
for i in range(0, 223):
    df = pd.read_csv(f'df_{i}.csv')
    def clean_event(event):
       event = re.sub('ts: \n  secs: \d+\n  nsecs:\s+\d+\n', '', event)
       return event
    df['events'] = df['events'].apply(clean_event)
    df.to_csv(f'df_{i}_split.csv', index=False)

In [179]:
for i in range(0, 233):
    df = pd.read_csv(f'df_{i}_split.csv')
    df = df['events'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('events')
    df.to_csv(f'df_{i}_split_rows.csv', index=False)

In [180]:
for i in range(0, 233):
    df = pd.read_csv(f'df_{i}_split_rows.csv')
    pattern = r"x: (?P<x>\d+)\ny: (?P<y>\d+)\npolarity: (?P<polarity>True|False)"
    df[['x', 'y', 'polarity']] = df['events'].str.extract(pattern)
    first_column = df.columns[0]
    df = df.drop(first_column, axis=1)
    df.to_csv(f'numerical_0_1/df_{i}_final.csv', index=False)

In [181]:
for i in range(0, 233):
    df = pd.read_csv(f'numerical_0_1/df_{i}_final.csv')
    img = np.zeros((640, 480))
    for j in range(0, df.shape[0]):
        if df.iloc[j, 2] == True:
            x = int(df.iloc[j, 0])
            y = int(df.iloc[j, 1])
            img[x, y] = 1
    img = (img * 255).astype(np.uint8)  
    cv2.imwrite(f'img_{i}.png', img)

In [196]:
# Median filter to remove noise
for i in range(0, 233):
    img = Image.open(f'raw_image/img_{i}.png')
    image_array = np.array(img)
    filtered_image_array = median_filter(image_array, size=7)
    white_pixel_indices = np.where(filtered_image_array == 255)
    
    if white_pixel_indices[0].size > 0:
        min_y, max_y = np.min(white_pixel_indices[0]), np.max(white_pixel_indices[0])
        min_x, max_x = np.min(white_pixel_indices[1]), np.max(white_pixel_indices[1])

        cv2.rectangle(image_array, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)

        filtered_image = Image.fromarray(image_array)
        filtered_image.save(f'annotated_image/img_{i}.png')
    else:
        img.save(f'annotated_image/img_{i}.png')
  

In [200]:
# Median filter and DBSCAN
for i in range(0, 233):
    img = Image.open(f'raw_image/img_{i}.png')
    image_array = np.array(img)
    filtered_image_array = median_filter(image_array, size=7)
    white_pixel_indices = np.where(filtered_image_array == 255)
    
    if white_pixel_indices[0].size > 0:
        data = np.array(list(zip(white_pixel_indices[1], white_pixel_indices[0])))

        dbscan = DBSCAN(eps=4, min_samples=2)
        dbscan.fit(data)

        unique_labels = set(dbscan.labels_)

        for label in unique_labels:
            cluster_indices = np.where(dbscan.labels_ == label)

            min_y, max_y = np.min(data[cluster_indices, 1]), np.max(data[cluster_indices, 1])
            min_x, max_x = np.min(data[cluster_indices, 0]), np.max(data[cluster_indices, 0])

            cv2.rectangle(image_array, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)

        filtered_image = Image.fromarray(image_array)
        filtered_image.save(f'cluster/img_{i}.png')
    else:
        img.save(f'cluster/img_{i}.png')

In [208]:
# Erosion and DBSCAN
for i in range(0, 233):
    img = Image.open(f'raw_image/img_{i}.png')
    image_array = np.array(img)
    eroded_image_array = erosion(image_array, square(4))
    
    white_pixel_indices = np.where(eroded_image_array == 255)
    
    if white_pixel_indices[0].size > 0:
        data = np.array(list(zip(white_pixel_indices[1], white_pixel_indices[0])))

        dbscan = DBSCAN(eps=4, min_samples=2)
        dbscan.fit(data)

        unique_labels = set(dbscan.labels_)

        for label in unique_labels:
            cluster_indices = np.where(dbscan.labels_ == label)

            min_y, max_y = np.min(data[cluster_indices, 1]), np.max(data[cluster_indices, 1])
            min_x, max_x = np.min(data[cluster_indices, 0]), np.max(data[cluster_indices, 0])

            cv2.rectangle(image_array, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)

        filtered_image = Image.fromarray(image_array)
        filtered_image.save(f'cluster/img_{i}.png')
    else:
        img.save(f'cluster/img_{i}.png')